## Indicadores de Salud de la Ciudad de México

In [1]:
if(! require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, readxl, janitor, stringr, writexl, foreign, downloader, tools)

Cargando paquete requerido: pacman



Descargar datos de Defunciones Regsitradas de INEGI, que es la que se toma de acuerdo con Secretaría de Salud

In [2]:
#Definiciones para antes de descargar los datos
# Crear directorio principal
dir_datos <- 'Estadísticas de Defunciones Registradas (EDR)/Microdatos_1'
if (!dir.exists(dir_datos)) {
  dir.create(dir_datos, recursive = TRUE)
}
print(file.exists(dir_datos)) #True = directorio creado

[1] TRUE


In [5]:
# Función para descargar y procesar los dbf de defunciones
descargar_defunciones_dbf <- function() {
  options(timeout = 90000)
  lista_defun <- list()
  
  # Lista de URLs con sus años o periodos
  urls <- list(
    '2023' = 'https://www.inegi.org.mx/contenidos/programas/edr/microdatos/defunciones/2023/defunciones_base_datos_2023_dbf.zip',
    '2022' = 'https://www.inegi.org.mx/contenidos/programas/edr/microdatos/defunciones/2022/defunciones_base_datos_2022_dbf.zip',
    '2021' = 'https://www.inegi.org.mx/contenidos/programas/edr/microdatos/defunciones/2021/defunciones_base_datos_2021_dbf.zip',
    '2020' = 'https://www.inegi.org.mx/contenidos/programas/edr/microdatos/defunciones/2020/defunciones_base_datos_2020_dbf.zip'
    #'2015_2019' = 'https://www.inegi.org.mx/contenidos/programas/edr/microdatos/defunciones/datos/defunciones_generales_base_datos_2015_2019_dbf.zip'
  )
  
  for (periodo in names(urls)) {
    url <- urls[[periodo]]
    zip_path <- file.path(dir_datos, basename(url))
    download.file(url, zip_path, mode = "wb")
    
    # Carpeta para extraer
    unzip_dir <- file.path(dir_datos, paste0('Defunciones_', periodo))
    dir.create(unzip_dir, showWarnings = FALSE)
    
    unzip(zip_path, exdir = unzip_dir)
    unlink(zip_path)  # borrar zip
    
    # Buscar archivos .dbf en el unzip_dir
    dbf_files <- list.files(unzip_dir, pattern = '\\.dbf$', full.names = TRUE, ignore.case = TRUE)
    
    if(length(dbf_files) == 0) {
      warning(paste0('No se encontró archivo DBF para ', periodo))
      next
    }
    
    # Leer cada DBF y guardarlo en lista
    period_lista_defun <- list()
    for(dbf_file in dbf_files) {
      nombre <- file_path_sans_ext(basename(dbf_file))
      df <- tryCatch({
        read.dbf(dbf_file, as.is = TRUE) %>% as_tibble() %>% clean_names()
      }, error = function(e) {
        warning(paste0('Error leyendo DBF: ', dbf_file))
        NULL
      })
      
      if(!is.null(df)) {
        period_lista_defun[[nombre]] <- df
      }
    }
    
    lista_defun[[periodo]] <- period_lista_defun
  }
  
  save(lista_defun, file = file.path(dir_datos, 'defunciones_lista_defun.RData'))
  return(lista_defun)
}

In [4]:
# Ejecutar descarga
defunciones <- descargar_defunciones_dbf()

: 

In [4]:
#Guardar la lista completa para no volver a descargar los datos
saveRDS(defunciones, file = 'Estadísticas de Defunciones Registradas (EDR)/edr_data.rds')

ERROR: Error: objeto 'defunciones' no encontrado


In [6]:
#Cargar la lista para evitar descargar los datos de nuevo
defunciones <- readRDS('Estadísticas de Defunciones Registradas (EDR)/edr_data.rds')

Descargar los datos del Subsistema de Información sobre Nacimientos (SINAC) de la Secretaría de Salud Federal.

*Comparando los nacimientos de la tabla de nacimientos de datos abiertos, con los de los tabulados de Secretaría de Salud, vemos que los datos ya están ajustados con CONAPO*

In [7]:
#Definiciones para antes de descargar los datos

#Crear lista para guardar los datos de sinac
lista_nacimientos <- list()

options(timeout = 600) #Aumentar el tiempo de espera para la descarga

In [8]:
#iterar por los años de descarga
    for(year in 19:23) {
#Crear la cadena de los años 2020 a 2023
        
        year_str <- paste0('20', year)
        #Cosntruir la url de descarga
        url <- paste0('http://www.dgis.salud.gob.mx/descargas/datosabiertos/nacimientos/sinac_', year_str, '.zip')
        
        #nombre y carpeta del archivo zip
        zip_file <- paste0('SINAC/sinac_', year_str, '.zip')
        extract_dir <- paste0('SINAC/sinac_', year_str)
#Crear carpeta si no existe
        if(!dir.exists('SINAC')) dir.create('SINAC')

#Intentar descargar archivos con trycatch
        tryCatch({
            download(url, destfile = zip_file, mode = 'wb')
            
            #Crear carpeta de extracción si no existe
            if(!file.exists(extract_dir)) dir.create(extract_dir)
                          
            #Descrompimir el archivo zip
            unzip(zip_file, exdir = extract_dir)

             #Buscar los archivos descomprimidos
            files <- list.files(extract_dir, full.names = TRUE)
    
    #Leer el primer archivo CSV encontrado (ajusta si es otro formato)
            csv_file <- files[grep("\\.csv$", files)][1]
    
        if (!is.na(csv_file)) {
        df <- read.csv(csv_file, stringsAsFactors = FALSE) %>% clean_names()
      
        # Guardar el dataframe en la lista
        lista_nacimientos[[year_str]] <- df
        
        # Eliminar el archivo ZIP
        file.remove(zip_file)
        
        message(paste('Año', year_str, 'procesado correctamente.'))
        } else {
        message(paste('No se encontró archivo CSV en', extract_dir))
        }
    }, error = function(e) {
        message(paste('Error al procesar el año', year_str))
    })
    }


Año 2019 procesado correctamente.

Año 2020 procesado correctamente.

Año 2021 procesado correctamente.

Año 2022 procesado correctamente.

Año 2023 procesado correctamente.



In [9]:
#Guardar la lista completa para no volver a descargar los datos
saveRDS(lista_nacimientos, file = 'SINAC/sinac_data.rds')

In [10]:
#Cargar la lista para evitar descargar los datos de nuevo
lista_nacimientos <- readRDS('SINAC/sinac_data.rds')

Descargar los datos del Censo Población y Vivienda 2020

In [192]:
# Crear lista para guardar el censo
lista_censo <- tibble()

# Definir el año (en este caso solo 2020)
anios_censo <- c(2020)

# Aumentar tiempo de espera
options(timeout = 600)

In [193]:
# Carpeta raíz donde se guardará todo
base_dir <- 'INEGI_CENSO'

# URL fija del ZIP del censo
url <- 'https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/Censo2020_CPV_CB_Personas_ejemplo_csv.zip'

In [194]:
# Carpeta raíz donde se guardará todo
base_dir <- 'INEGI_CENSO'

# Carpeta donde se descomprimirá el archivo (con el nombre que tú quieres)
extract_dir <- file.path(base_dir, 'INEGI. Censo 2020')

# URL fija del ZIP del censo
url <- 'https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/Censo2020_CPV_CB_Personas_ejemplo_csv.zip'

# Archivo ZIP con nombre fijo
zip_file <- file.path(base_dir, 'censo_2020.zip')

# Crear carpeta base si no existe
if (!dir.exists(base_dir)) dir.create(base_dir)

# Descargar ZIP (solo si no existe)
if (!file.exists(zip_file)) {
  download.file(url, destfile = zip_file, mode = 'wb')
}

# Descomprimir directamente en la carpeta con el nombre que deseas
if (!dir.exists(extract_dir)) {
  unzip(zip_file, exdir = extract_dir)
}

# Listar todos los archivos descomprimidos para verificar
files_all <- list.files(extract_dir, full.names = TRUE, recursive = TRUE)
print('Archivos descomprimidos:')
print(files_all)

# Buscar archivo CSV (sin importar mayúsculas/minúsculas)
files_csv <- list.files(extract_dir, pattern = "\\.[cC][sS][vV]$", full.names = TRUE, recursive = TRUE)

if (length(files_csv) > 0) {
  csv_file <- files_csv[1]  # Tomamos el primer CSV que encuentre
  censo_df <- read.csv(csv_file, stringsAsFactors = FALSE) %>% janitor::clean_names()
  
  message(paste('Censo 2020 leído correctamente desde:', csv_file))
} else {
  message('⚠️ No se encontró ningún archivo CSV en la carpeta descomprimida.')
}


[1] "Archivos descomprimidos:"
[1] "INEGI_CENSO/INEGI. Censo 2020/Censo2020_CPV_CB_Personas_ejemplo_csv.CSV"


Censo 2020 leído correctamente desde: INEGI_CENSO/INEGI. Censo 2020/Censo2020_CPV_CB_Personas_ejemplo_csv.CSV



In [196]:
str(censo_df)

'data.frame':	1257506 obs. of  43 variables:
 $ ent             : int  21 21 21 14 14 14 14 21 21 21 ...
 $ mun             : int  108 108 108 86 86 86 44 86 216 216 ...
 $ loc             : int  1 1 1 1 1 1 1 1 1 1 ...
 $ ageb            : chr  "0017" "0017" "0017" "0017" ...
 $ mza             : int  1 1 1 3 3 3 3 4 6 6 ...
 $ seg             : chr  "N" "N" "N" "N" ...
 $ id_viv          : num  2.11e+11 2.11e+11 2.11e+11 1.41e+11 1.41e+11 ...
 $ id_persona      : num  2.11e+16 2.11e+16 2.11e+16 1.41e+16 1.41e+16 ...
 $ tipo_reg        : int  0 0 0 0 0 0 0 0 0 0 ...
 $ clase_viv       : int  2 2 2 3 3 3 3 2 2 2 ...
 $ numper          : int  1 3 2 3 1 2 1 1 1 2 ...
 $ sexo            : int  1 3 1 3 3 1 1 3 3 1 ...
 $ edad            : int  35 66 78 2 18 21 28 39 70 47 ...
 $ parent          : int  1 6 6 3 1 2 1 1 1 3 ...
 $ ent_pais_nac    : int  32 32 32 1 1 1 1 20 5 1 ...
 $ afrodes         : int  3 3 3 3 3 3 3 3 3 3 ...
 $ dhsersal1       : int  1 9 9 1 1 1 9 1 1 9 ...
 $ dhsersal2 

Datos de población de CONAPO

In [177]:
# Vector de años (ejemplo, aunque el ZIP no varía con el año)
anios <- 2023  # o la lista que tengas

# Carpeta raíz donde se guardará todo
base_dir <- 'CONAPO'


# URL fija del ZIP (mismo siempre)
url <- 'https://conapo.segob.gob.mx/work/models/CONAPO/pry23/DB/ConDem50a19_ProyPob20a70.zip'



In [175]:
# Crear lista para guardar los datos de CONAPO
lista_conapo <- tibble()

In [180]:
# URL fija del ZIP
url <- 'https://conapo.segob.gob.mx/work/models/CONAPO/pry23/DB/ConDem50a19_ProyPob20a70.zip'

# Archivos y carpetas fijos según el ZIP descargado
zip_file <- file.path(base_dir, 'conapo_data.zip')
extract_dir <- file.path(base_dir, 'ConDem50a19_ProyPob20a70')

# Crear carpeta base si no existe
if (!dir.exists(base_dir)) dir.create(base_dir)

# Descargar y descomprimir (solo una vez)
if (!file.exists(zip_file)) {
  download.file(url, destfile = zip_file, mode = 'wb')
}

if (!dir.exists(extract_dir)) {
  unzip(zip_file, exdir = base_dir)
}

# Buscar archivo Excel dentro de la carpeta descomprimida
files <- list.files(extract_dir, pattern = "\\.xlsx$", full.names = TRUE)
excel_file <- files[1]  # El primer archivo Excel que encuentre

if (!is.na(excel_file)) {
  # Leer el Excel directamente como tibble
  conapo_df <- read_excel(excel_file) %>% janitor::clean_names()
  
  message('Archivo CONAPO leído correctamente.')
} else {
  message('No se encontró archivo Excel dentro de la carpeta descomprimida.')
}


Archivo CONAPO leído correctamente.



In [191]:
str(conapo_df)

tibble [744,920 × 7] (S3: tbl_df/tbl/data.frame)
 $ renglon  : num [1:744920] 1 2 3 4 5 6 7 8 9 10 ...
 $ ano      : num [1:744920] 1950 1950 1950 1950 1950 1950 1950 1950 1950 1950 ...
 $ entidad  : chr [1:744920] "República Mexicana" "República Mexicana" "República Mexicana" "República Mexicana" ...
 $ cve_geo  : num [1:744920] 0 0 0 0 0 0 0 0 0 0 ...
 $ edad     : num [1:744920] 0 0 1 1 2 2 3 3 4 4 ...
 $ sexo     : chr [1:744920] "Hombres" "Mujeres" "Hombres" "Mujeres" ...
 $ poblacion: num [1:744920] 577250 564734 520692 511958 487298 ...


Definimos los años para los cálculos

In [128]:
anios <- 2019:2023

Tasa de Mortalidad en menores de 5 años de la Ciudad de México

In [143]:
#Conjunto de nombres para los dataframes de defunciones
nombres_def <- c('2020' = 'defun20','2021' = 'defun21', '2022' = 'DEFUN22', '2023' = 'DEFUN23')

In [144]:
#Crear dataframe vacío para almacenar los resultados
mortalidad_infantil <- tibble(anio = integer(), nacimientos = integer(), defunciones = integer(), tasa = numeric())


In [145]:
# Iterar sobre los años y calcular tasas
for (a in anios) {
  anio_str <- as.character(a)
  
  if (!is.null(lista_nacimientos[[anio_str]]) && !is.null(defunciones[[anio_str]])) {
    df_nac <- lista_nacimientos[[anio_str]]
    nombre_df <- nombres_def[anio_str]
    df_def <- defunciones[[anio_str]][[nombre_df]]
    
    nac <- df_nac %>%
      filter(entidadresidencia == 9) %>%
      summarise(nacimientos = n()) %>%
      pull(nacimientos)
    
    def <- df_def %>%
      filter(ent_resid == '09', anio_ocur == a, edad >= 1001, edad < 4005) %>%
      summarise(defunciones = n()) %>%
      pull(defunciones)
    
    tasa <- def / nac * 1000
    
    mortalidad_infantil <- mortalidad_infantil %>% 
    add_row(anio = a, nacimientos = nac, defunciones = def, tasa = tasa)
  } else {
    message(paste('Datos incompletos para el año', a))
  }
}

print(mortalidad_infantil)

Datos incompletos para el año 2019



# A tibble: 4 × 4
   anio nacimientos defunciones  tasa
  <int>       <int>       <int> <dbl>
1  2020       87005        1266  14.6
2  2021       77476        1062  13.7
3  2022       80892        1159  14.3
4  2023       78032        1152  14.8


Total de nacimientos ocurridos en niñas y adolescentes menores de 15 años 2019-2023

*Secretaría de Salud utiliza la variable Entidad de federativa del parto/certificado*

In [128]:
#Crear tabla para guiardar los  resultados
madres_menores_15 <- tibble(anio = integer(), nacimientos = integer())

In [130]:
for(a in anios) {
    anio_str <- as.character(a)

    if(!is.null(lista_nacimientos[[anio_str]])) {
        df_nac <- lista_nacimientos[[anio_str]] %>% as.tibble()
        
        #Definir las variables Entidad y Edad por año
        if (a == 2019) {
            var_entidad <- 'ent_cert'
            var_edad <- 'edadm'
        } else {
           var_entidad <- 'entidadfederativacertifica'
           var_edad <- 'edad'
        }
        
        #Filtrar y calcular
        nacimientos_15 <- df_nac %>%
        filter(.data [[var_entidad]] == 9, .data[[var_edad]] < 15) %>%
        summarise(nacimientos = n()) %>%
        pull(nacimientos)
        
        #Guardar los resutlados en la tabla creada
        madres_menores_15 <- madres_menores_15 %>%
        add_row(anio = a, nacimientos = nacimientos_15)
        
    } else {
        message(paste('Datos incompletos para el año', a))
    }
}

#Ver resultados
print(madres_menores_15)

# A tibble: 5 × 2
   anio nacimientos
  <int>       <int>
1  2019         322
2  2020         272
3  2021         321
4  2022         304
5  2023         299


Mortalidad por enfermedades del corazón

*De acuerdo con INEGI, se clasifica la mortalidad por enfermedades del corazón (con excepción del paro cardiaco) utilizando el conjunto de códigos de la Lista Mexicana de Enfermedades del 26 al 29.*

In [135]:
#Revisamos la Lista Mexicana de Enfermedades (ya incluida en la descarga de los datos de Defunción) 
#filtramos los códigos de las enfermedades del corazón, que utilizaremos como criterio de selección en los datos de defunciones 

causa_def <- as_tibble(defunciones$'2020'$LISTAMEX) %>%
filter(cve %in% c('26', '26A', '26B', '27', '27A', '27B', '27Z', 
                     '28', '28A', '28Z', '29', '29A', '29B', '29C', '29D', '29Z'))

causa_def

Warning message in FUN(X[[i]], ...):
"input string 2 is invalid in this locale"
Warning message in FUN(X[[i]], ...):
"input string 2 is invalid in this locale"
Warning message in FUN(X[[i]], ...):
"input string 2 is invalid in this locale"
Warning message in FUN(X[[i]], ...):
"input string 2 is invalid in this locale"
Warning message in FUN(X[[i]], ...):
"input string 2 is invalid in this locale"
Warning message in FUN(X[[i]], ...):
"input string 2 is invalid in this locale"
Warning message in FUN(X[[i]], ...):
"input string 2 is invalid in this locale"
Warning message in FUN(X[[i]], ...):
"input string 2 is invalid in this locale"
Warning message in FUN(X[[i]], ...):
"input string 2 is invalid in this locale"
Warning message in FUN(X[[i]], ...):
"input string 2 is invalid in this locale"
Warning message in FUN(X[[i]], ...):
"input string 2 is invalid in this locale"
Warning message in FUN(X[[i]], ...):
"input string 2 is invalid in this locale"
Warning message in FUN(X[[i]], ...):
"in

cve,descrip
<chr>,<chr>
26,Fiebre reum�tica aguda y enfermedades card�acas reum�ticas cr�nicas
26A,Fiebre reum�tica aguda
26B,Enfermedades reum�ticas cr�nicas del coraz�n
27,Enfermedades hipertensivas
27A,Enfermedad card�aca hipertensiva
27B,Hipertensi�n esencial
27Z,Las dem�s enfermedades hipertensivas
28,Enfermedades isqu�micas del coraz�n
28A,Infarto agudo del miocardio


In [157]:
#Crear un dataframe para guardar los datos
defun_cora <- tibble(anio = integer(), defunciones = integer())

In [158]:
for (a in anios) {
  anio_str <- as.character(a)
  
  if (!is.null(defunciones[[anio_str]])) {
    # Buscar el nombre de la tabla de defunciones
    nombres_tablas <- names(defunciones[[anio_str]])
    tabla_defun <- nombres_tablas[str_detect(nombres_tablas, regex('defun', ignore_case = TRUE))][1]
    
    if (is.na(tabla_defun)) {
      message(paste('No se encontró tabla de defunciones para el año', a))
      next
    }
    
    df_defun <- defunciones[[anio_str]][[tabla_defun]] %>% as_tibble()
    
    # Buscar el nombre de la variable de entidad
    var_entidad <- if ("ent_resid" %in% names(df_defun)) {
      "ent_resid"
    } else if ("entidad_residencia" %in% names(df_defun)) {
      "entidad_residencia"
    } else {
      stop(paste("No se encontró variable de entidad para el año", a))
    }
    
    # Filtrar y calcular
    defunciones_anio <- df_defun %>%
      filter(.data[[var_entidad]] == '09', lista_mex %in% enfer_cora) %>%  # usa la variable flexible
      summarise(defunciones = n()) %>%
      pull(defunciones)
    
    # Guardar resultados
    defun_cora <- defun_cora %>%
      add_row(anio = a, defunciones = defunciones_anio)
    
  } else {
    message(paste('Datos incompletos para el año', a))
  }
}

# Ver resultados
print(defun_cora)


Datos incompletos para el año 2019



# A tibble: 4 × 2
   anio defunciones
  <int>       <int>
1  2020       21064
2  2021       18610
3  2022       16810
4  2023       17020


In [189]:
# Crear tabla de resultados
defun_cora <- tibble()


In [197]:
for (a in anios) {
  anio_str <- as.character(a)
  
  if (!is.null(defunciones[[anio_str]])) {
    nombres_tablas <- names(defunciones[[anio_str]])
    tabla_defun <- nombres_tablas[str_detect(nombres_tablas, regex('defun', ignore_case = TRUE))][1]
    
    if (is.na(tabla_defun)) {
      message(paste('No se encontró tabla de defunciones para el año', a))
      next
    }
    
    df_defun <- defunciones[[anio_str]][[tabla_defun]] %>% as_tibble()
    
    var_entidad <- if ("ent_resid" %in% names(df_defun)) {
      "ent_resid"
    } else if ("entidad_residencia" %in% names(df_defun)) {
      "entidad_residencia"
    } else {
      stop(paste("No se encontró variable de entidad para el año", a))
    }
    
    defunciones_anio <- df_defun %>%
      filter(.data[[var_entidad]] == '09', lista_mex %in% enfer_cora) %>%
      summarise(defunciones = n()) %>%
      pull(defunciones)
    
    # Población del año correspondiente
    if (a == 2020) {
      poblacion_anio <- censo_df %>%
        filter(ent == 9) %>%
        summarise(poblacion = sum(fac)) %>%  # Aquí usa el nombre real de tu factor de expansión
        pull(poblacion)
    } else {
      poblacion_anio <- conapo_df %>%
        filter(entidad == "Ciudad de México", ano == a, edad == 0) %>%  # edad 0 es el total consolidado
        summarise(poblacion = sum(poblacion)) %>%  # suma hombres y mujeres
        pull(poblacion)
    }
    
    tasa <- (defunciones_anio / poblacion_anio) * 100000
    
    defun_cora <- defun_cora %>%
      add_row(anio = a, defunciones = defunciones_anio, poblacion = poblacion_anio, tasa = tasa)
    
  } else {
    message(paste('Datos incompletos para el año', a))
  }
}


ERROR: [1m[33mError[39m in `add_row()`:[22m
[1m[22m[33m![39m New rows can't add columns.
[31m✖[39m Can't find columns `anio`, `defunciones`, `poblacion`, and `tasa` in `.data`.
